In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gsm8k-modified/main_test.csv
/kaggle/input/gsm8k-modified/main_train.csv
/kaggle/input/gsm8k-modified/socratic_train.csv
/kaggle/input/gsm8k-modified/socratic_test.csv


In [2]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

In [3]:
df = pd.read_csv("/kaggle/input/gsm8k-modified/main_test.csv")

In [4]:
df.head()

,question,answer,value
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20


In [5]:
# Step 2: Load the Qwen2.5-Math-1.5B-Instruct model
model_name = "Qwen/Qwen2.5-Math-1.5B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def extract_answer(response):
    """Extracts the answer from model response using regex."""
    match = re.search(r"\\boxed{(\d+)}", response)
    return int(match.group(1)) if match else None

correct = 0


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [6]:
def get_model_answer(question):
    messages = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user", "content": question}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return extract_answer(response)

# Step 3: Iterate through each question and compute accuracy
total = len(df)
for index, row in df.iterrows():
    question = row["question"]
    true_value = row["value"]
    model_answer = get_model_answer(question)
    print(model_answer)
    if model_answer == true_value:
        correct += 1
    
accuracy = (correct / total) * 100
print(f"Model Accuracy: {accuracy:.2f}%")

18
3
120000
180
20
64
None
140
315
460
366
694
12
18
60
96
230
57500
7
6
15
23
7
8
26
2
243
16
25
104
109
80
35
70
23
9
75
0
10
18
8
200
26
48
None
104
163
800
8
30
294
None
15
40
40
14
3
83
None
187
17
1430
2500
1596
300
36
48
595
36
60
7425
60
221
255
88
60
5
100
6
70
10
17
623
600
15
44
22
None
8000
18
225
28
4
None
348
40
3
12
5
40
695
None
15
140
500
20
72
3
50
28
45
16
24
25
6
0
42
360
4
None
240
27
48
50
10
10
82
120
880
10000
30
940
60
13
720
40
6
None
105
70
20
400
140
None
20
4000
2125
60
30
16
4
None
4
48
272
280
1400
80
34
3
16
32
None
54
105
77
5
16
18
120
150
1210
43
18000
95
15
100
450
122
130
20
160
23
2
25
30
None
None
50
34
360
5
91
24
10
None
120
6277
320
None
55
109000
100
31
98
110
860
2600
76
155
10
4
5
250
8
44
220
15
45
54
70
90
140
20000
180
18
33
None
1
21
276000
50
75
12
28
10
31
90
68
280
21
6
3
250
20
7
27000
32
300
5600
17
70
73
18
84
192
45
5600
None
168
11
62
270
None
400
9500
182000
63
1375
4
762
20
None
315
3200
138
9
4
40
6
7
2450
195
68
360
21
90
8
3

## Model Accuracy: 81.43%